In [1]:
import torch
import numpy as np
from collections import deque
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn  # Make sure to import nn module

# Define the LSTM model as before
class LSTMModel(nn.Module):
    def __init__(self, input_size=3, hidden_size=50, output_size=3 * 20):
        super(LSTMModel, self).__init__()
        self.lstm1 = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.lstm2 = nn.LSTM(hidden_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x, _ = self.lstm1(x)
        x, _ = self.lstm2(x)
        x = x[:, -1, :]  # Get the output of the last time step
        x = self.fc(x)
        return x

# Initialize model, criterion, and optimizer
input_length = 80
output_length = 20
model = LSTMModel()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Function to generate non-fault periodic vibrational data
def generate_non_fault_data(time_steps=100, noise_level=0.1):
    t = np.linspace(0, 10, time_steps)
    frequency = 0.5
    x = np.sin(2 * np.pi * frequency * t) + noise_level * np.random.randn(time_steps)
    y = np.sin(2 * np.pi * frequency * t + np.pi / 4) + noise_level * np.random.randn(time_steps)
    z = np.sin(2 * np.pi * frequency * t + np.pi / 2) + noise_level * np.random.randn(time_steps)
    return np.stack([x, y, z], axis=-1)

# Prepare dataset with targets being the same as inputs
def prepare_dataset(num_samples=500):
    inputs = []
    targets = []
    for _ in range(num_samples):
        # Generate non-fault data
        non_fault_data = generate_non_fault_data(input_length + output_length)

        # Split into input and target sequences
        inputs.append(non_fault_data[:input_length])  # Input part
        targets.append(non_fault_data[input_length:])  # Target part (the next part of the same data)

    # Convert lists to tensors
    inputs = torch.tensor(inputs, dtype=torch.float32)
    targets = torch.tensor(targets, dtype=torch.float32).reshape(num_samples, -1)  # Flatten targets
    return TensorDataset(inputs, targets)

# Training loop
def train_model(model, criterion, optimizer, num_epochs=50, batch_size=32):
    dataset = prepare_dataset()
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    for epoch in range(num_epochs):
        epoch_loss = 0.0
        for inputs, targets in dataloader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()

        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss / len(dataloader):.4f}")

# Run training
train_model(model, criterion, optimizer)

# Save the model state dictionary locally
model_path = "./lstm_model.pth"
torch.save(model.state_dict(), model_path)
print(f"Model saved to {model_path}")


/var/folders/2w/cxhzvjr95x3gvgqw7pmmsklr0000gn/T/ipykernel_12561/1607744094.py:51: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:277.)
  inputs = torch.tensor(inputs, dtype=torch.float32)


Epoch [1/50], Loss: 0.4432
Epoch [2/50], Loss: 0.1452
Epoch [3/50], Loss: 0.0185
Epoch [4/50], Loss: 0.0121
Epoch [5/50], Loss: 0.0104
Epoch [6/50], Loss: 0.0101
Epoch [7/50], Loss: 0.0101
Epoch [8/50], Loss: 0.0101
Epoch [9/50], Loss: 0.0101
Epoch [10/50], Loss: 0.0101
Epoch [11/50], Loss: 0.0101
Epoch [12/50], Loss: 0.0101
Epoch [13/50], Loss: 0.0101
Epoch [14/50], Loss: 0.0101
Epoch [15/50], Loss: 0.0101
Epoch [16/50], Loss: 0.0101
Epoch [17/50], Loss: 0.0101
Epoch [18/50], Loss: 0.0101
Epoch [19/50], Loss: 0.0101
Epoch [20/50], Loss: 0.0101
Epoch [21/50], Loss: 0.0101
Epoch [22/50], Loss: 0.0101
Epoch [23/50], Loss: 0.0101
Epoch [24/50], Loss: 0.0101
Epoch [25/50], Loss: 0.0101
Epoch [26/50], Loss: 0.0101
Epoch [27/50], Loss: 0.0101
Epoch [28/50], Loss: 0.0101
Epoch [29/50], Loss: 0.0101
Epoch [30/50], Loss: 0.0101
Epoch [31/50], Loss: 0.0101
Epoch [32/50], Loss: 0.0101
Epoch [33/50], Loss: 0.0101
Epoch [34/50], Loss: 0.0101
Epoch [35/50], Loss: 0.0101
Epoch [36/50], Loss: 0.0101
E